<a href="https://colab.research.google.com/github/pranjalchaubey/Predicting-Stock-Direction/blob/master/src/Stock_Direction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Direction
In this notebook we are going to use Random Forests to predict the direction  
of stocks for the next trading day.  
In more technical terms, we are going to try and predict the _**Momentum**_  
of the stocks.  
Based on the momentum, we are going to give the following signals to the user  
1. Buy (_+ve momentum_)
2. Sell (_-ve momentum_)
3. Hold (_neutral_)  

This system will work on the daily closing prices and will generate the signals by predicting the momentum for the next trading day. 

##Setup The Colab Environment 
Install and Import the required libraries.  

In [0]:
# Debug Flag
_GLOBAL_DEBUG_ = True

In [2]:
# Install the yFinance library 
!pip install yfinance 

  Created wheel for yfinance: filename=yfinance-0.1.45-cp36-none-any.whl size=14652 sha256=851469314c4a23d274bd47dd3ab8f2f36e265a5fa702321f6317f327f2981ff8
  Stored in directory: /root/.cache/pip/wheels/0c/d1/df/aa9a7744a4ac353cc9a1f2c3aaea7c1f457fc49de4286f2d88
Successfully built yfinance


In [0]:
# yFinance will help us fetch the data for our dataset
import yfinance as yf

In [0]:
# Data Mining and plotting 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
# Classifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [0]:
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import os
import sys
import shutil
# if in Google Colaboratory
try:
    from google.colab import drive
except:
    pass

import warnings
warnings.filterwarnings("ignore")

In [0]:
# Configure Pandas Display Options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [8]:
# Since I am running this notebook on Colab, 
# Let's try and get some system information 
import platform
print('System Processor: ', platform.processor(), '\n')
!nvidia-smi

System Processor:  x86_64 

Mon Sep 30 12:39:47 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                

## Load the Data
We will extract the 5 year history of 10 stocks chosen by the user. 

<br/>Here, we will take the following stocks as an example
1. Facebook FB
2. Apple AAPL
3. Amazon AMZN
4. Netflix NFLX
5. Google GOOGL
6. Starbucks SBUX
7. Exxon Mobil XOM
8. Johnson & Johnson JNJ
9. Bank of America BAC
10. General Motors GM

<br/>In the actual application, stock selection will be done by the user. 

In [0]:
# List of stock tickers 
# this info will come from the user, perhaps in the form of a pickle file 
ticker_list = ['FB', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'SBUX', 'XOM', 'JNJ',\
               'BAC', 'GM']

In [0]:
def fetch_ticker_data(tickers): 
    """
    Fetch 5 years of historical data for the 'tickers', from Yahoo Finance.    

    Parameters
    ----------
    tickers : List   
        List of stocks chosen by the user.   
        
    Returns
    -------
    data : Dataframe 
        Pandas df with historical stock data time series. 
    """
    # Debug 
    _LOCAL_DEBUG_ = False

    # Get the Data from Yahoo! Finance
    data = yf.download(tickers, start="2017-01-01", end="2017-09-07")
    if _GLOBAL_DEBUG_ and _LOCAL_DEBUG_:
        display(data.head(20))

    # Data is multi-indexed on the columns 
    # Make it multi-index on the rows, to make it 
    # fit for consumption by the RandomForestClassiffier
    data = data.stack(1) 
    if _GLOBAL_DEBUG_ and _LOCAL_DEBUG_:
        display(data.head(20))

    # Drop all the columns except for Adj Close & Volume
    data = data[['Adj Close', 'Volume']]
    # Rename the column names 
    data.columns = ['close', 'volume']  
    if _GLOBAL_DEBUG_ and _LOCAL_DEBUG_:
        display(data.head(20))

    return data

In [110]:
# Go fetch! 
dataset = fetch_ticker_data(tickers=ticker_list)

[*********************100%***********************]  10 of 10 downloaded


## Data Preprocessing
Now that we have the data downloaded and neatly organized in a dataframe,  
time to do some necessary preprocessing.  
In particular, we are going to use the _log returns_ of the stocks to create a  
target column, called `target`.  
The target variable will be created using the following criteria  
```
  -1 = Sell = ret < -0.0015
   0 = Hold = -0.0015 < ret < 0.0015 
   1 = Buy  = ret > .0015
```

In [0]:
def classify_return(ret):
    """
    Classify the returns as -1, 0 or 1.
    -1 = Sell = ret < -0.0015
    0 = Hold = -0.0015 < ret < 0.0015 
    1 = Buy  = ret > .0015

    Parameters
    ----------
    ret : Float   
        Stock return for the current day, d. 
        retd = log(retd) - log(ret(d-1))

    Returns
    -------
    ret_category : Integer
        Category of return, ret. 
        -1, 0 or 1. 
    """
    if ret < -0.0015: 
        ret_category = -1
    elif -0.0015 < ret and ret < 0.0015:
        ret_category = 0
    elif ret > 0.0015:
        ret_category = 1

    return ret_category

In [0]:
def data_preprocess(dataset):
    """
    Calculates log returns and creates the 'target' column.

    Parameters
    ----------
    dataset : Pandas Dataframe
    Dataframe containing price, volume and other 
    information of the chosen stocks. 

    Returns
    -------
    Processed dataframe with two additional columns,
    'returns' and 'target'. 
    returns = log returns of prices 
    target = target column for the classifier to predict
    """
    # Debug Flag
    _LOCAL_DEBUG_ = False 

    # Advance the prices by 1 day to calculate the log returns 
    dataset['shift'] = dataset.groupby(level=1)['close'].shift(1)
    if _GLOBAL_DEBUG_ and _LOCAL_DEBUG_:
        dataset.head(30)

    # Create the 'returns' column
    dataset['returns'] = np.log(dataset['close']) - np.log(dataset['shift'])
    if _GLOBAL_DEBUG_ and _LOCAL_DEBUG_:
        dataset.head(30)   

    # Drop the first row as it contains NANs in the returns column
    dataset.drop(index = dataset.index.levels[0].values[0], level=0,\
                 inplace=True)
    if _GLOBAL_DEBUG_ and _LOCAL_DEBUG_:
        dataset.head(30)

    # Create the 'target' column for the classifier 
    dataset['target'] = dataset['returns'].apply(lambda x: classify_return(x))
    if _GLOBAL_DEBUG_ and _LOCAL_DEBUG_:
        dataset.head(30)

    # Drop the 'shift' column as it was only temporary to calculate 'returns'
    dataset.drop(['shift'], axis=1, inplace=True)
    if _GLOBAL_DEBUG_ and _LOCAL_DEBUG_:
        dataset.head(30)

    return dataset

Pre-process the downloaded dataset. 

In [113]:
display(dataset.head(20))
dataset = data_preprocess(dataset)
display(dataset.head(20))

close    volume
Date                              
2017-01-03 AAPL   111.29  28781900
           AMZN   753.67   3521100
           BAC     21.41  99298100
           FB     116.86  20663900
           GM      31.13  10904900
           GOOGL  808.01   1959000
           JNJ    107.65   5953000
           NFLX   127.49   9437900
           SBUX    52.39   7809300
           XOM     81.20  10360600
2017-01-04 AAPL   111.16  21118100
           AMZN   757.18   2510500
           BAC     21.81  76875100
           FB     118.69  19630900
           GM      32.85  23388500
           GOOGL  807.77   1515300
           JNJ    107.48   5828900
           NFLX   129.41   7843600
           SBUX    53.00   7796300
           XOM     80.30   9434200

close    volume   returns  target
Date                                                
2017-01-04 AAPL   111.16  21118100 -0.001169       0
           AMZN   757.18   2510500  0.004646       1
           BAC     21.81  76875100  0.018510       1
           FB     118.69  19630900  0.015538       1
           GM      32.85  23388500  0.053780       1
           GOOGL  807.77   1515300 -0.000297       0
           JNJ    107.48   5828900 -0.001580      -1
           NFLX   129.41   7843600  0.014948       1
           SBUX    53.00   7796300  0.011576       1
           XOM     80.30   9434200 -0.011146      -1
2017-01-05 AAPL   111.73  22193600  0.005115       1
           AMZN   780.45   5830100  0.030270       1
           BAC     21.55  86826400 -0.011993      -1
           FB     120.67  19492200  0.016544       1
           GM      32.23  15636700 -0.019054      -1
           GOOGL  813.02   1340500  0.006478       1
           JNJ    108.60   6217200  0.010367       1
           NFLX   131.81  10185500  0.018376       1
           SBUX    53.44   7602300  0.008268       1
           XOM     79.11  14443200 -0.014930      -1

## Feature Engineering 
Leaving this section blank at the moment. Will come back at a later time. 

## Train Test Split 
Split the data into testing and training sets.  
In production, there's going to be only re-training of the model that we have  
chosen with the specified hyper-parameters. 

In [0]:
def train_test_split(dataset, features, target, train_size, test_size):
    """
    Generate the train and test dataset.

    Parameters
    ----------
    dataset : DataFrame
        All the samples including target
    features : List
        List of the names of columns that are features
    target : String
        Name of column that is the target (in our case, 'target')
    train_size : float
        The proportion of the data used for the training dataset
    test_size : float
        The proportion of the data used for the test dataset

    Returns
    -------
    x_train : DataFrame
        The train input samples
    x_test : DataFrame
        The test input samples
    y_train : Pandas Series
        The train target values
    y_test : Pandas Series
        The test target values
    """
    # Data Sanity check 
    assert train_size >= 0 and train_size <= 1.0, 'Train size out of bounds!'
    assert test_size >= 0 and test_size <= 1.0, 'Test size out of bounds!'
    assert train_size + test_size == 1.0, 'Train + Test should be equal to 1!'
    
    # Debug Flag 
    _LOCAL_DEBUG_ = False
    
    # Extract the x and y from the dataset
    all_x = dataset[features]
    all_y = dataset[target]
    if _GLOBAL_DEBUG_ and _LOCAL_DEBUG_:
        print(all_x.head())
        print(all_y.head())
        print('dataset.shape ', dataset.shape)

    # Get the number of rows in df and no of elements in the pandas series 
    # NOTE - Both are multi-indexed
    len_x = len(all_x.index.levels[0])
    len_y = len(all_y.index.levels[0])
    if _GLOBAL_DEBUG_ and _LOCAL_DEBUG_:
        print('Len x ', len_x)
        print('Len y ', len_y)
    
    # Some fancy calculations here for x
    x_train = all_x.loc[all_x.index.levels[0]\
                        [:int(len_x*train_size)].astype(str).tolist()]
    x_test = all_x.loc[all_x.index.levels[0]\
                       [int(len_x*train_size):int(len_x*train_size) + int(len_x*test_size)]\
                       .astype(str).tolist()]
    if _GLOBAL_DEBUG_ and _LOCAL_DEBUG_:
        print('x_train.shape ', x_train.shape)
        print('x_test.shape ', x_test.shape)

    # Some fancy calculations here for y as well
    y_train = all_y.loc[all_y.index.levels[0]\
                        [:int(len_y*train_size)].astype(str).tolist()]
    y_test = all_y.loc[all_y.index.levels[0]\
                       [int(len_y*train_size):int(len_y*train_size) + int(len_y*test_size)]\
                       .astype(str).tolist()]
    if _GLOBAL_DEBUG_ and _LOCAL_DEBUG_:
        print('y_train.shape ', y_train.shape)
        print('y_test.shape ', y_test.shape)
 
    return x_train, x_test, y_train, y_test 

In [0]:
# In our dataset all the columns except for the 'target' column are features
# Temporarily drop the 'target' and extract the features  
features = dataset.drop(['target'], axis=1).columns.values.tolist()

# Get the train and test sets 
X_train, X_test, y_train, y_test = train_test_split(dataset=dataset,\
                                                    features=features,\
                                                    target='target',\
                                                    train_size=0.9,\
                                                    test_size=0.1)

In [140]:
# Validate the results 
print('dataset.shape ', dataset.shape)
print('X_train.shape ', X_train.shape)
print('X_test.shape ', X_test.shape)
print('y_train.shape ', y_train.shape)
print('y_test.shape ', y_test.shape)

dataset.shape  (1700, 4)
X_train.shape  (1520, 3)
X_test.shape  (170, 3)
y_train.shape  (1520,)
y_test.shape  (170,)
